In [3]:
# calculate the frequency and wav length in record level
import contextlib
import wave
import os
from tqdm import tqdm

file_path_wav = '../dataset/train_wav/'
record_names = [os.path.join(file_path_wav, f) for f in os.listdir(file_path_wav)]
record_num = len(record_names)
print(record_num)

count = 0
count0 = 0
count1 = 0
count2 = 0
count3 = 0
for i, record_name in enumerate(tqdm(record_names)):
    with contextlib.closing(wave.open(record_name, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        wav_length = frames / float(rate)
        if rate == 8000:
            count+=1
        else:
            print("frequency：",rate,"HZ")
            print(record_name)
            count0+=1            
        if wav_length == 9.216:
            count1+=1
        elif wav_length == 15.36:
            count2+=1
        else:
            print("length：",wav_length,"s")
            print(record_name)
            count3+=1
print("count: "+ str(count))
print("count0: "+ str(count0))
print("count1: "+ str(count1))
print("count2: "+ str(count2))
print("count3: "+ str(count3))

1950


 75%|███████▌  | 1463/1950 [00:01<00:00, 1291.55it/s]

length： 8.512 s
../dataset/train_wav/41103864_7.6_1_p1_1387.wav


100%|██████████| 1950/1950 [00:01<00:00, 1346.37it/s]

count: 1950
count0: 0
count1: 1430
count2: 519
count3: 1


In [2]:
# divide the training and validation set for task2_wav
import os
import shutil

file_path_task2_wav_src = '../dataset/train_wav/'
file_path_task2_wav = './task2_wav/train/'
save_path_task2_val = './task2_wav/val/'

def copy_allfiles(src, dest):
    src_files = os.listdir(src)
    for file_name in src_files:
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dest)

copy_allfiles(file_path_task2_wav_src, file_path_task2_wav)

record_names = [f for f in os.listdir(file_path_task2_wav)]
record_names = sorted(record_names)
record_num = len(record_names)
print(record_num)

for i in range(int(0.9*record_num), record_num):
    shutil.move(file_path_task2_wav+record_names[i], save_path_task2_val+record_names[i])

record_name_train = [f for f in os.listdir(file_path_task2_wav)]
record_num_train = len(record_name_train)
print(record_num_train)

record_name_val = [s for s in os.listdir(save_path_task2_val)]
record_num_val = len(record_name_val)
print(record_num_val)

1950
1755
195


In [3]:
# generate the responding training set for task1_wav
# generate the responding training annotations for task1_wav and task2_wav
from tqdm import tqdm
import json
import os
from pydub import AudioSegment

file_path_task2_wav = './task2_wav/train/'
file_path_json = '../dataset/train_json/'
save_path_task1_wav = './task1_wav/train/'
save_path_task11_json = './input/train/task11_input.json'
save_path_task12_json = './input/train/task12_input.json'
save_path_task21_json = './input/train/task21_input.json'
save_path_task22_json = './input/train/task22_input.json'

record_names = [f for f in os.listdir(file_path_task2_wav)]
record_names = sorted(record_names)
record_num = len(record_names)
print("The number of task2 wav is: %d" % record_num)

task11_dic = {}
task12_dic = {}
task21_dic = {}
task22_dic = {}
num = 0

for i, record_name in enumerate(tqdm(record_names)):
    record_json_name = record_name[:-4]+'.json'
    record_json = os.path.join(file_path_json, record_json_name)
    if os.path.exists(record_json):
        with open(record_json, 'rb') as f:
            item = json.load(f)
            num+=1
            # task22 annotations
            task22_dic[record_name] = item['record_annotation']
            # task21 annotations
            if item['record_annotation'] == "Normal" or item['record_annotation'] == "Poor Quality":
                task21_dic[record_name] = item['record_annotation']
            else:
                task21_dic[record_name] = "Adventitious"

            if item['event_annotation']:
                sound = AudioSegment.from_wav(file_path_task2_wav+record_name)
                for j, event_item in enumerate(item['event_annotation']):
                    # save task1 wav
                    start_time = int(event_item['start'])
                    end_time = int(event_item['end'])
                    event_sound = sound[start_time:end_time]
                    event_name = record_name[:-4]+'_'+str(j)+'.wav'
                    event_sound.export(save_path_task1_wav+event_name, format="wav")
                    # task12 annotations
                    task12_dic[event_name] = event_item['type']
                    # task11 annotations
                    if event_item['type'] == "Normal":
                        task11_dic[event_name] = event_item['type']
                    else:
                        task11_dic[event_name] = "Adventitious"
    else:
        print(record_json_name + "does not exist!")

event_names = [f for f in os.listdir(save_path_task1_wav)]
event_num = len(event_names)
print("The number of task1 wav is: %d" % event_num)
print("The number of json file is: %d" % num)
print("The number of task11 annotations is: %d" % len(task11_dic))
print("The number of task12 annotations is: %d" % len(task12_dic))
print("The number of task21 annotations is: %d" % len(task21_dic))
print("The number of task22 annotations is: %d" % len(task22_dic))

# save task11 annotations         
with open(save_path_task11_json, 'w') as f11:
    json.dump(task11_dic, f11)
# save task12 annotations         
with open(save_path_task12_json, 'w') as f12:
    json.dump(task12_dic, f12)
# save task21 annotations         
with open(save_path_task21_json, 'w') as f21:
    json.dump(task21_dic, f21)
# save task22 annotations         
with open(save_path_task22_json, 'w') as f22:
    json.dump(task22_dic, f22)

The number of task2 wav is: 1755


100%|██████████| 1755/1755 [00:20<00:00, 84.06it/s] 


The number of task1 wav is: 5914
The number of json file is: 1755
The number of task11 annotations is: 5914
The number of task12 annotations is: 5914
The number of task21 annotations is: 1755
The number of task22 annotations is: 1755


In [4]:
# generate the responding validation set for task1_wav
# generate the responding validation annotations for task1_wav and task2_wav
from tqdm import tqdm
import json
import os
from pydub import AudioSegment

file_path_task2_wav = './task2_wav/val/'
file_path_json = '../dataset/train_json/'
save_path_task1_wav = './task1_wav/val/'
save_path_task11_json = './input/val/task11_input.json'
save_path_task12_json = './input/val/task12_input.json'
save_path_task21_json = './input/val/task21_input.json'
save_path_task22_json = './input/val/task22_input.json'

record_names = [f for f in os.listdir(file_path_task2_wav)]
record_names = sorted(record_names)
record_num = len(record_names)
print("The number of task2 wav is: %d" % record_num)

task11_dic = {}
task12_dic = {}
task21_dic = {}
task22_dic = {}
num = 0

for i, record_name in enumerate(tqdm(record_names)):
    record_json_name = record_name[:-4]+'.json'
    record_json = os.path.join(file_path_json, record_json_name)
    if os.path.exists(record_json):
        with open(record_json, 'rb') as f:
            item = json.load(f)
            num+=1
            # task22 annotations
            task22_dic[record_name] = item['record_annotation']
            # task21 annotations
            if item['record_annotation'] == "Normal" or item['record_annotation'] == "Poor Quality":
                task21_dic[record_name] = item['record_annotation']
            else:
                task21_dic[record_name] = "Adventitious"

            if item['event_annotation']:
                sound = AudioSegment.from_wav(file_path_task2_wav+record_name)
                for j, event_item in enumerate(item['event_annotation']):
                    # save task1 wav
                    start_time = int(event_item['start'])
                    end_time = int(event_item['end'])
                    event_sound = sound[start_time:end_time]
                    event_name = record_name[:-4]+'_'+str(j)+'.wav'
                    event_sound.export(save_path_task1_wav+event_name, format="wav")
                    # task12 annotations
                    task12_dic[event_name] = event_item['type']
                    # task11 annotations
                    if event_item['type'] == "Normal":
                        task11_dic[event_name] = event_item['type']
                    else:
                        task11_dic[event_name] = "Adventitious"
    else:
        print(record_json_name + " does not exist!")

event_names = [f for f in os.listdir(save_path_task1_wav)]
event_num = len(event_names)
print("The number of task1 wav is: %d" % event_num)
print("The number of json file is: %d" % num)
print("The number of task11 annotations is: %d" % len(task11_dic))
print("The number of task12 annotations is: %d" % len(task12_dic))
print("The number of task21 annotations is: %d" % len(task21_dic))
print("The number of task22 annotations is: %d" % len(task22_dic))

# save task11 annotations         
with open(save_path_task11_json, 'w') as f11:
    json.dump(task11_dic, f11)
# save task12 annotations         
with open(save_path_task12_json, 'w') as f12:
    json.dump(task12_dic, f12)
# save task21 annotations         
with open(save_path_task21_json, 'w') as f21:
    json.dump(task21_dic, f21)
# save task22 annotations         
with open(save_path_task22_json, 'w') as f22:
    json.dump(task22_dic, f22)

The number of task2 wav is: 195


100%|██████████| 195/195 [00:02<00:00, 84.48it/s]

test1.json does not exist!
test2.json does not exist!
The number of task1 wav is: 742
The number of json file is: 193
The number of task11 annotations is: 742
The number of task12 annotations is: 742
The number of task21 annotations is: 193
The number of task22 annotations is: 193


In [4]:
# calculate the number of all classes in training set of task11 
from tqdm import tqdm
import json
import torch

file_path = './input/train/task11_input.json'
num_0 = 0
num_1 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Adventitious":
            num_1+=1
        else:
            print(value)

num = num_0 + num_1
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Adventitious\" is: %d" % num_1)
print("The number of all classes is: %d" % num)
task11_class_weight = []
task11_class_weight.append(num/num_0)
task11_class_weight.append(num/num_1)
print(f'The task11 class weight is: {task11_class_weight}')
class_weights = torch.FloatTensor(task11_class_weight)
print(class_weights)

100%|██████████| 5914/5914 [00:00<00:00, 1279405.50it/s]

The number of class "Normal" is: 4555
The number of class "Adventitious" is: 1359
The number of all classes is: 5914
The task11 class weight is: [1.2983534577387486, 4.351729212656365]
tensor([1.2984, 4.3517])


In [6]:
# calculate the number of all classes in validation set of task11 
from tqdm import tqdm
import json

file_path = './input/val/task11_input.json'
num_0 = 0
num_1 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Adventitious":
            num_1+=1
        else:
            print(value)

num = num_0 + num_1
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Adventitious\" is: %d" % num_1)
print("The number of all classes is: %d" % num)

100%|██████████| 744/744 [00:00<00:00, 916733.89it/s]

The number of class "Normal" is: 605
The number of class "Adventitious" is: 139
The number of all classes is: 744


In [6]:
# calculate the number of all classes in training set of task12
from tqdm import tqdm
import json

file_path = './input/train/task12_input.json'
num_0 = 0
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0
num_5 = 0
num_6 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Rhonchi":
            num_1+=1
        elif value == "Wheeze":
            num_2+=1
        elif value == "Stridor":
            num_3+=1
        elif value == "Coarse Crackle":
            num_4+=1
        elif value == "Fine Crackle":
            num_5+=1
        elif value == "Wheeze+Crackle":
            num_6+=1
        else:
            print(value)

num = num_0 + num_1 + num_2 + num_3 + num_4 + num_5 + num_6
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Rhonchi\" is: %d" % num_1)
print("The number of class \"Wheeze\" is: %d" % num_2)
print("The number of class \"Stridor\" is: %d" % num_3)
print("The number of class \"Coarse Crackle\" is: %d" % num_4)
print("The number of class \"Fine Crackle\" is: %d" % num_5)
print("The number of class \"Wheeze+Crackle\" is: %d" % num_6)
print("The number of all classes is: %d" % num)
task12_class_weight = []
task12_class_weight.append(num/num_0)
task12_class_weight.append(num/num_1)
task12_class_weight.append(num/num_2)
task12_class_weight.append(num/num_3)
task12_class_weight.append(num/num_4)
task12_class_weight.append(num/num_5)
task12_class_weight.append(num/num_6)
print(f'The task12 class weight is: {task12_class_weight}')
class_weights = torch.FloatTensor(task12_class_weight)
print(class_weights)

100%|██████████| 5914/5914 [00:00<00:00, 605549.25it/s]

The number of class "Normal" is: 4555
The number of class "Rhonchi" is: 39
The number of class "Wheeze" is: 400
The number of class "Stridor" is: 14
The number of class "Coarse Crackle" is: 49
The number of class "Fine Crackle" is: 830
The number of class "Wheeze+Crackle" is: 27
The number of all classes is: 5914
The task12 class weight is: [1.2983534577387486, 151.64102564102564, 14.785, 422.42857142857144, 120.6938775510204, 7.125301204819277, 219.03703703703704]
tensor([  1.2984, 151.6410,  14.7850, 422.4286, 120.6939,   7.1253, 219.0370])


In [8]:
# calculate the number of all classes in validation set of task12
from tqdm import tqdm
import json

file_path = './input/val/task12_input.json'
num_0 = 0
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0
num_5 = 0
num_6 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Rhonchi":
            num_1+=1
        elif value == "Wheeze":
            num_2+=1
        elif value == "Stridor":
            num_3+=1
        elif value == "Coarse Crackle":
            num_4+=1
        elif value == "Fine Crackle":
            num_5+=1
        elif value == "Wheeze+Crackle":
            num_6+=1
        else:
            print(value)

num = num_0 + num_1 + num_2 + num_3 + num_4 + num_5 + num_6
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Rhonchi\" is: %d" % num_1)
print("The number of class \"Wheeze\" is: %d" % num_2)
print("The number of class \"Stridor\" is: %d" % num_3)
print("The number of class \"Coarse Crackle\" is: %d" % num_4)
print("The number of class \"Fine Crackle\" is: %d" % num_5)
print("The number of class \"Wheeze+Crackle\" is: %d" % num_6)
print("The number of all classes is: %d" % num)

100%|██████████| 744/744 [00:00<00:00, 643414.88it/s]

The number of class "Normal" is: 605
The number of class "Rhonchi" is: 0
The number of class "Wheeze" is: 52
The number of class "Stridor" is: 1
The number of class "Coarse Crackle" is: 0
The number of class "Fine Crackle" is: 83
The number of class "Wheeze+Crackle" is: 3
The number of all classes is: 744


In [7]:
# calculate the number of all classes in training set of task21 
from tqdm import tqdm
import json

file_path = './input/train/task21_input.json'
num_0 = 0
num_1 = 0
num_2 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Adventitious":
            num_1+=1
        elif value == "Poor Quality":
            num_2+=1
        else:
            print(value)

num = num_0 + num_1 + num_2
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Adventitious\" is: %d" % num_1)
print("The number of class \"Poor Quality\" is: %d" % num_2)
print("The number of all classes is: %d" % num)
task21_class_weight = []
task21_class_weight.append(num/num_0)
task21_class_weight.append(num/num_1)
task21_class_weight.append(num/num_2)
print(f'The task21 class weight is: {task21_class_weight}')
class_weights = torch.FloatTensor(task21_class_weight)
print(class_weights)

100%|██████████| 1755/1755 [00:00<00:00, 1072719.84it/s]

The number of class "Normal" is: 1164
The number of class "Adventitious" is: 436
The number of class "Poor Quality" is: 155
The number of all classes is: 1755
The task21 class weight is: [1.5077319587628866, 4.025229357798165, 11.32258064516129]
tensor([ 1.5077,  4.0252, 11.3226])


In [10]:
# calculate the number of all classes in validation set of task21 
from tqdm import tqdm
import json

file_path = './input/val/task21_input.json'
num_0 = 0
num_1 = 0
num_2 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "Adventitious":
            num_1+=1
        elif value == "Poor Quality":
            num_2+=1
        else:
            print(value)

num = num_0 + num_1 + num_2
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"Adventitious\" is: %d" % num_1)
print("The number of class \"Poor Quality\" is: %d" % num_2)
print("The number of all classes is: %d" % num)

100%|██████████| 195/195 [00:00<00:00, 679874.71it/s]

The number of class "Normal" is: 140
The number of class "Adventitious" is: 34
The number of class "Poor Quality" is: 21
The number of all classes is: 195


In [8]:
# calculate the number of all classes in training set of task22
from tqdm import tqdm
import json

file_path = './input/train/task22_input.json'
num_0 = 0
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "CAS":
            num_1+=1
        elif value == "DAS":
            num_2+=1
        elif value == "CAS & DAS":
            num_3+=1
        elif value == "Poor Quality":
            num_4+=1
        else:
            print(value)

num = num_0 + num_1 + num_2 + num_3 + num_4
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"CAS\" is: %d" % num_1)
print("The number of class \"DAS\" is: %d" % num_2)
print("The number of class \"CAS & DAS\" is: %d" % num_3)
print("The number of class \"Poor Quality\" is: %d" % num_4)
print("The number of all classes is: %d" % num)
task22_class_weight = []
task22_class_weight.append(num/num_0)
task22_class_weight.append(num/num_1)
task22_class_weight.append(num/num_2)
task22_class_weight.append(num/num_3)
task22_class_weight.append(num/num_4)
print(f'The task22 class weight is: {task22_class_weight}')
class_weights = torch.FloatTensor(task22_class_weight)
print(class_weights)

100%|██████████| 1755/1755 [00:00<00:00, 1191679.38it/s]

The number of class "Normal" is: 1164
The number of class "CAS" is: 124
The number of class "DAS" is: 234
The number of class "CAS & DAS" is: 78
The number of class "Poor Quality" is: 155
The number of all classes is: 1755
The task22 class weight is: [1.5077319587628866, 14.153225806451612, 7.5, 22.5, 11.32258064516129]
tensor([ 1.5077, 14.1532,  7.5000, 22.5000, 11.3226])


In [12]:
# calculate the number of all classes in validation set of task22
from tqdm import tqdm
import json

file_path = './input/val/task22_input.json'
num_0 = 0
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0

with open(file_path, 'rb') as f:
    items = json.load(f)
    for value in tqdm(items.values()):
        if value == "Normal":
            num_0+=1
        elif value == "CAS":
            num_1+=1
        elif value == "DAS":
            num_2+=1
        elif value == "CAS & DAS":
            num_3+=1
        elif value == "Poor Quality":
            num_4+=1
        else:
            print(value)

num = num_0 + num_1 + num_2 + num_3 + num_4
print("The number of class \"Normal\" is: %d" % num_0)
print("The number of class \"CAS\" is: %d" % num_1)
print("The number of class \"DAS\" is: %d" % num_2)
print("The number of class \"CAS & DAS\" is: %d" % num_3)
print("The number of class \"Poor Quality\" is: %d" % num_4)
print("The number of all classes is: %d" % num)

100%|██████████| 195/195 [00:00<00:00, 329528.32it/s]

The number of class "Normal" is: 140
The number of class "CAS" is: 10
The number of class "DAS" is: 15
The number of class "CAS & DAS" is: 9
The number of class "Poor Quality" is: 21
The number of all classes is: 195
